# Robot leg kinematics

In [1]:
import numpy as np

import matplotlib as mpl
mpl.use('Qt5Agg')

import matplotlib.pyplot as plt
plt.ion()

# for the symbolic manipulation of jacobian
import sympy as sp
# from sympy import symbols
# from sympy import sin, cos, asin, acos, pi, atan2, sqrt
from sympy.utilities.lambdify import lambdify
# from sympy import Matrix

from scipy.optimize import minimize
from scipy.optimize import fsolve

In [2]:
## Motor constants
K = 0.0285;                 # Nm / A
peak_amp = 30;              # A
peak_torque = K * peak_amp; # Nm
m_motor = 0.2;              # kg
m_assembly = 0.2;           # kg

m_total = 2*m_motor + m_assembly; # kg

gravity = 9.8;              # m/s^2

weight = m_total * gravity  # N

l1 = 0.09;                  # m 
l2 = 0.16;                  # m
w = 0.07;                   # m


## Closed chain forward kinematics

First we define the leg geometry and derive the loop equations which represents the closed chain constraint on the 5-bar. 

![](leg_kinematics_2019.png)



Define our symbolic variables

In [3]:
# solve jacobian of constraint equation
(thetaL_sym, 
 thetaR_sym, 
 link1_sym, 
 link2_sym, 
 width_sym) = sp.symbols("""thetaL_sym 
                            thetaR_sym 
                            link1_sym 
                            link2_sym 
                            width_sym""", real = True)


## Let's first derive the forwards kinematics of this robot leg

The challenge here is to derive the forward kinematics *without* needing to know the internal angles. We can do this in the following way, we first solve for the "tilt" angle $\theta_3$ and the base width $L$. We can then construct a series of transformations which take us to the toe position. 

![](FK_derivation.png)


In [4]:
def T(theta, x, y):
    """
    Function to return an arbitrary transformation matrix 
    This is for sympy symbolic calculation
    """
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), x], 
                      [sp.sin(theta), sp.cos(theta), y],
                      [0, 0, 1]])

def sym_to_np(T):
    return np.array(T).astype(np.float64)



## FK Through transformation matrices

In [5]:
x_r = width_sym/2 + link1_sym*sp.cos(thetaR_sym)
x_l = -width_sym/2 + link1_sym*sp.cos(thetaL_sym)

y_r = link1_sym*sp.sin(thetaR_sym)
y_l = link1_sym*sp.sin(thetaL_sym)

theta3_sym = sp.atan2(y_r - y_l, x_r - x_l)
L = sp.sqrt((x_l - x_r)**2 + (y_l - y_r)**2)

FK = T(thetaL_sym, -width_sym/2, 0)@T(-(thetaL_sym - theta3_sym), link1_sym, 0)@sp.Matrix([L/2, sp.sqrt(link2_sym**2 - (L/2)**2), 1])
FK = FK[:2,:]
FK.simplify()
FK

Matrix([
[                                                                  link1_sym*((sin(thetaL_sym) - sin(thetaR_sym))*sqrt(2*link1_sym**2*cos(thetaL_sym - thetaR_sym) - 2*link1_sym**2 + 2*link1_sym*width_sym*cos(thetaL_sym) - 2*link1_sym*width_sym*cos(thetaR_sym) + 4*link2_sym**2 - width_sym**2)/sqrt(-2*link1_sym**2*cos(thetaL_sym - thetaR_sym) + 2*link1_sym**2 - 2*link1_sym*width_sym*cos(thetaL_sym) + 2*link1_sym*width_sym*cos(thetaR_sym) + width_sym**2) + cos(thetaL_sym) + cos(thetaR_sym))/2],
[link1_sym*(-sin(thetaL_sym) + sin(thetaR_sym))/2 + link1_sym*sin(thetaL_sym) + (-link1_sym*cos(thetaL_sym) + link1_sym*cos(thetaR_sym) + width_sym)*sqrt(2*link1_sym**2*cos(thetaL_sym - thetaR_sym) - 2*link1_sym**2 + 2*link1_sym*width_sym*cos(thetaL_sym) - 2*link1_sym*width_sym*cos(thetaR_sym) + 4*link2_sym**2 - width_sym**2)/(2*sqrt(-2*link1_sym**2*cos(thetaL_sym - thetaR_sym) + 2*link1_sym**2 - 2*link1_sym*width_sym*cos(thetaL_sym) + 2*link1_sym*width_sym*cos(thetaR_sym) + width_sym**2))

In [6]:
xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w), (thetaL_sym, 0/2), (thetaR_sym, np.pi/2)])
xy



# xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w), (thetaR_sym, thetaR), (thetaL_sym, thetaL)]).evalf()

Matrix([
[-0.104567022159455],
[0.0117628839645655]])

## FK Through FSolve method

In [7]:
from scipy.optimize import fsolve 

def constraint_eqn(xy_points, theta_R, theta_L, l1, l2, w):
    x_pt = xy_points[0]
    y_pt = xy_points[1]
    
    return (l1**2 - l2**2 + (x_pt - w/2)**2 + y_pt**2 - 2*l1 * (y_pt * np.sin(theta_R) + (x_pt - w/2)*np.cos(theta_R)),
            l1**2 - l2**2 + (x_pt + w/2)**2 + y_pt**2 - 2*l1 * (y_pt * np.sin(theta_L) + (x_pt + w/2)*np.cos(theta_L)))


thetaR = .5
thetaL = np.pi

out = fsolve(lambda x: constraint_eqn(x, thetaR, thetaL, l1, l2,w), (0, 0.1))

print(out)


[-0.02402171  0.12411037]


In [8]:
constraint_of_x = lambda x: constraint_eqn(x, thetaR, thetaL, l1, l2,w)


constraint_of_x(out)


(-3.599551212651875e-17, -3.859759734048396e-17)

## Double check the FK

In [9]:
thetaR = np.pi/2
thetaL = np.pi/2

# Using forward kinematics 
xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w), (thetaR_sym, thetaR), (thetaL_sym, thetaL)]).evalf()


# Using fsolve
out = fsolve(lambda x: constraint_eqn(x, thetaR, thetaL, l1, l2, w), (0, -0.1))


print("Compare FK computed using the fsolve (top) and the direct (bottom) method: \r\n fzero --> x = {} \t y = {} \r\n Symb ---> x = {} \t y = {}".format(out[0], out[1], xy[0], xy[1]))

Compare FK computed using the fsolve (top) and the direct (bottom) method: 
 fzero --> x = -1.4919511438410443e-18 	 y = -0.06612494995995996 
 Symb ---> x = 5.51091059616309E-18 	 y = 0.246124949959960


In [10]:
def internal_angles(thetaR, thetaL, l1 = l1, l2 = l2, w = w):

    # 
    # system_of_equations = @(x) sum(abs([w + l1*(cos(b1)-cos(a1)) + l2*(cos(x(2))-cos(x(1)));
    #                                    l1*(sin(b1)-sin(a1)) + l2*(sin(x(2))-sin(x(1)))]));
    # 
    # x_guess = [a1, b1 + pi/2];                               
    # % [x,fval,exitflag,output] = fminsearch(system_of_equations, x_guess);
    # [x,fval,exitflag,output] = fmincon(system_of_equations, ...
    #                                    x_guess, ...
    #                                    [], [], [], [], ...
    #                                    [-pi, -pi], [pi, pi]);
    # 
    # a2 = x(1);
    # b2 = x(2);
    # 
    # x = w/2 + l1*cos(b1) + l2*cos(b2);
    # y = l1*sin(b1) + l2*sin(b2);

    def sys(x): 
        return (w + l1*np.cos(thetaR) + l2*np.cos(x[0]) - l1*np.cos(thetaL) - l2*np.cos(x[1]),
                l1*np.sin(thetaR) + l2*np.sin(x[0]) - l1*np.sin(thetaL) - l2*np.sin(x[1]))

    alphaR, alphaL = fsolve(sys, (np.pi/2, np.pi/2))
    
    alphaR = alphaR % (2*np.pi)
    alphaL = alphaL % (2*np.pi)
        
    # Copmute FK for checking
    x = w/2 + l1*np.cos(thetaR) + l2*np.cos(alphaR);
    y = l1*np.sin(thetaR) + l2*np.sin(alphaR);

    return (alphaR, alphaL, x, y)

thetaR = .5
thetaL = np.pi

(alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL)

# Should produce
# alphaL
# Out[17]: 0.8878073988680342

# alphaR
# Out[18]: 2.611036674795031


## Double check the FK

In [11]:
thetaR = .2
thetaL = np.pi

# Using fzero method
(alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL)

# Using forward kinematics 
xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w), (thetaR_sym, thetaR), (thetaL_sym, thetaL)]).evalf()

print("Compare FK computed using the fsolve (top) and the direct (bottom) method: \r\n x = {} \t y = {} \r\n x = {} \t y = {}".format(x, y, xy[0], xy[1]))

Compare FK computed using the fsolve (top) and the direct (bottom) method: 
 x = -0.008124629215593038 	 y = 0.10927098290031101 
 x = -0.00812462921557965 	 y = 0.109270982900330


Define a helper function to draw the current robot

In [12]:
def draw_robot(thetaR, thetaL, link1 = l1, link2 = l2, width = w, ax = False):
    
    # Solve for internal angles
    (alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL)

    def pol2cart(rho, phi):
        x = rho * np.cos(phi)
        y = rho * np.sin(phi)
        return(x, y)

    if ax == False:
        ax = plt.gca()
        ax.cla()
    

    ax.plot(-width/2, 0, 'ok')
    ax.plot(width/2, 0, 'ok')

    ax.plot([-width/2, 0], [0, 0], 'k')
    ax.plot([width/2, 0], [0, 0], 'k')
    
    ax.plot(-width/2 + np.array([0, link1*np.cos(thetaL)]), [0, link1*np.sin(thetaL)], 'k')
    ax.plot(width/2 + np.array([0, link1*np.cos(thetaR)]), [0, link1*np.sin(thetaR)], 'k')

    ax.plot(-width/2 + link1*np.cos(thetaL) + np.array([0, link2*np.cos(alphaL)]), \
             link1*np.sin(thetaL) + np.array([0, link2*np.sin(alphaL)]), 'k');
    
    ax.plot(width/2 + link1*np.cos(thetaR) + np.array([0, link2*np.cos(alphaR)]), \
             np.array(link1*np.sin(thetaR) + np.array([0, link2*np.sin(alphaR)])), 'k');
    
    ax.plot(x, y, 'ro');


#     plt.plot(x_end, y_end, 'go');

    ax.axis([-.3,.3,-.1,.3])
    
thetaR = .55
thetaL = np.pi

draw_robot(thetaR, thetaL)

## Now lets implement IK just to have that taken care of

We will use both the fzero method and the the optimize function of scipy to solve for the inverse kinematics, and to bias solutions to keep legs in the appropriate configuration. This works by choosing an optimization function to minimize subject to the IK constraint function. We 

## IK Through FZero method

In [65]:
from scipy.optimize import fsolve 

def IK_constraint_eqn(x, x_pt, y_pt, l1, l2, w):
    theta_R = x[0]
    theta_L = x[1]
    
    return (l1**2 - l2**2 + (x_pt - w/2)**2 + y_pt**2 - 2*l1 * (y_pt * np.sin(theta_R) + (x_pt - w/2)*np.cos(theta_R)),
            l1**2 - l2**2 + (x_pt + w/2)**2 + y_pt**2 - 2*l1 * (y_pt * np.sin(theta_L) + (x_pt + w/2)*np.cos(theta_L)))


x_foot = 0.05
y_foot = 0.1


# 0,np.pi

angles = fsolve(lambda x: IK_constraint_eqn(x, x_foot, y_foot, l1, l2,w), (0,np.pi))

print(angles)

plt.clf()
plt.plot(x_foot, y_foot, 'o', markersize = 20)
draw_robot(angles[0], angles[1], ax = plt.gca())



[-0.56007415  2.44873959]


In [69]:
def IK_5_link(x, y, l1 = l1, l2 = l2, w = w):
    
    def leg_wide(var):
        return np.linalg.norm([var[1] - np.pi, var[0]])
    
    def x_constraint_equation(var):
        # should be equal to zero when the 
        return l1**2 - l2**2 + (x - w/2)**2 + y**2 - 2*l1*(y*np.sin(var[0]) + (x - w/2)*np.cos(var[0]))

    def y_constraint_equation(var):
        return l1**2 - l2**2 + (x + w/2)**2 + y**2 - 2*l1*(y*np.sin(var[1]) + (x + w/2)*np.cos(var[1]))

    
    res = minimize(leg_wide, (0.1, 9*np.pi/10), method="SLSQP", constraints= ({"type": "eq", "fun": x_constraint_equation}, 
                                                                               {"type": "eq", "fun": y_constraint_equation}))
    
    return (res, np.linalg.norm([x_constraint_equation(res.x), y_constraint_equation(res.x)]))


# Test, the following theta's correspond to the x-y below
thetaR = .5
thetaL = np.pi
    
x = -0.024021708847354217
y = 0.12411037295149752

res = IK_5_link(x, y)


print("""Compare the FK position (top) and the IK solution (bottom) method: 
          \r\n theta_R = {:.4f} \t theta_L = {:.4f} \r\n theta_R = {:.4f} \t theta_L = {:.4f}""".format(thetaR, thetaL, res[0].x[0], res[0].x[1]))

Compare the FK position (top) and the IK solution (bottom) method: 
          
 theta_R = 0.5000 	 theta_L = 3.1416 
 theta_R = 0.5000 	 theta_L = 3.1416


## Now lets visualize the IK

In [67]:
# x_loc = -0.14;
# y_loc = 0.02;


x_loc = 0.1;
y_loc = 0.1;

res = IK_5_link(x_loc, y_loc)
thetaR = res[0].x[0]
thetaL = res[0].x[1]

draw_robot(thetaR, thetaL)
plt.plot(x_loc, y_loc, 'og', ms = 1)


## click points to visualize

In [70]:
thetaR = .5
thetaL = np.pi

fig, ax = plt.subplots()
draw_robot(thetaR, thetaL, ax = ax)

def onclick(event):
    res = IK_5_link(event.xdata, event.ydata)
    thetaR = res[0].x[0]
    thetaL = res[0].x[1]
    
    ax.cla()
    draw_robot(thetaR, thetaL, ax = ax)
    plt.show()
    plt.draw()
    
    plt.title("theta_R = {:.4f}, theta_L = {:.4f}, err = {:.4f}".format(thetaR, thetaL, res[1]))

   
cid = fig.canvas.mpl_connect('button_press_event', onclick)

# def onmotion(event):
#     res = IK_5_link(event.xdata, event.ydata)
#     thetaR = res[0].x[0]
#     thetaL = res[0].x[1]
    
#     ax.cla()
#     draw_robot(thetaR, thetaL, ax = ax)
#     plt.show()
#     plt.draw()
    
#     plt.title("theta_R = {:.4f}, theta_L = {:.4f}, err = {:.4f}".format(thetaR, thetaL, res[1]))
  
# cid = fig.canvas.mpl_connect('motion_notify_event', onmotion)


In [12]:
import pyglet

def plot(l1 = l1, l2 = l2, w = w): 
  
    l1 = l1*2000
    l2 = l2*2000
    w = w*2000
    
    # make our window for drawin'
    window = pyglet.window.Window()
 
    label = pyglet.text.Label('Mouse (x,y)', font_name='Times New Roman', 
        font_size=36, x=window.width//2, y=window.height//2,
        anchor_x='center', anchor_y='center')
 
    def get_joint_positions(thetaR, thetaL):
        """This method finds the (x,y) coordinates of each joint"""
        (alphaR, alphaL, x, y) = internal_angles(thetaR, thetaL, l1 = l1, l2 = l2, w = w)
        
        
        x = np.array([-w/2, -w/2 + l1*np.cos(thetaL), x, w/2+l1*np.cos(thetaR), w/2, -w/2]) + window.width/2
        y = np.array([0, l1*np.sin(thetaL), y, l1*np.sin(thetaR), 0, 0])
        
        return np.array([x, y]).astype('int')
     
    window.jps = get_joint_positions(0,0)
 
    @window.event
    def on_draw():
        window.clear()
        label.draw()
        for i in range(4): 
            pyglet.graphics.draw(2, pyglet.gl.GL_LINES, ('v2i', 
                (window.jps[0][i], window.jps[1][i], 
                 window.jps[0][i+1], window.jps[1][i+1])))
 
    @window.event
    def on_mouse_motion(x, y, dx, dy):
        # call the inverse kinematics function of the arm
        # to find the joint angles optimal for pointing at 
        # this position of the mouse 
        label.text = '(x,y) = (%.3f, %.3f)'%(x,y)
        
        res = IK_5_link(x - window.width/2, y)
        thetaR = res[0].x[0]
        thetaL = res[0].x[1]

        res = IK_5_link(x - window.width/2, y, l1 = l1, l2 = l2, w = w) # get new arm angles
        window.jps = get_joint_positions(res[0].x[0], res[0].x[1]) # get new joint (x,y) positions
 
    pyglet.app.run()
 
plot(l1 = l1, l2 = l2, w = w)

In [63]:
%qtconsole

# HW 2 solutions

# OK all looks good, now lets solve for the Jacobian and compute the design metrics

The Jacobian maps the two joint velocities to world velocity



In [20]:
xy = FK.subs([(link1_sym, l1), (link2_sym, l2), (width_sym, w)]).evalf()

J = xy.jacobian([thetaR_sym, thetaL_sym]).evalf()
J = lambdify((thetaR_sym, thetaL_sym), J)

First thing lets examine the manipulability metrics

In [22]:
x = 0
y = 0.2

res = IK_5_link(x, y)
thetaR = res[0].x[0]
thetaL = res[0].x[1]

fig, ax = plt.subplots()
draw_robot(thetaR, thetaL, ax = ax)

def onclick(event):
    x = event.xdata
    y = event.ydata

    res = IK_5_link(x, y)
    thetaR = res[0].x[0]
    thetaL = res[0].x[1]

    J_current = J(thetaR,thetaL)

    u, s, v = np.linalg.svd(J_current)

    ax.cla()
    draw_robot(thetaR, thetaL, ax = ax)

    ang = 180*np.arctan2(u[1,0], u[0,0])/np.pi
    ell = mpl.patches.Ellipse(xy=(x,y), width=s[0], height=s[1], angle = ang, alpha = 0.5)
    ax.add_patch(ell)

    plt.quiver(x, y, J_current[0,0], J_current[1,0], scale = 1, color = 'r')

    plt.quiver(x, y, J_current[0,1], J_current[1,1], scale = 1, color = 'b')

    #     plt.quiver(x, y, s[0]*u[0,0], s[0]*u[1,0], scale = 1, color = 'r')
    #     plt.quiver(x, y, -s[0]*u[0,0], -s[0]*u[1,0], scale = 1, color = 'r')

    #     plt.quiver(x, y, s[1]*u[0,1], s[1]*u[1,1], scale = 1, color = 'b')
    #     plt.quiver(x, y, -s[1]*u[0,1], -s[1]*u[1,1], scale = 1, color = 'b')


    plt.show()
    plt.draw()

    plt.title("theta_R = {:.4f}, theta_L = {:.4f}, err = {:.4f}".format(thetaR, thetaL, res[1]))

   
cid = fig.canvas.mpl_connect('button_press_event', onclick)



In [ ]:
q1 = np.linspace(-np.pi, np.pi, 100)
q2 = np.linspace(-np.pi, np.pi, 100)

workspace = np.zeros((q1.shape[0], q2.shape[0], 2))


for j, q1_ang in enumerate(q1):
    for k, q2_ang in enumerate(q2):

        workspace[j,k, :] = sym_to_np(FK.subs([(thetaL_sym, q1_ang), 
                                                (thetaR_sym, q2_ang),
                                                (link1_sym, l1), 
                                                (link2_sym, l2), 
                                                (width_sym, w)]).evalf()).T
        
    print(j)
        

## Plot out the (approximate) workspace

In [27]:
plt.clf()
plt.plot(np.ravel(workspace[:,:,0]), np.ravel(workspace[:,:,1]), 'o')
plt.xlabel('x (cm)')
plt.ylabel('y (cm)')
plt.gca().set_aspect('equal')

## One shot compute quantities

In [45]:
x = 0
y = 0.2

res = IK_5_link(x, y)
thetaR = res[0].x[0]
thetaL = res[0].x[1]

fig, ax = plt.subplots()
draw_robot(thetaR, thetaL, ax = ax)


res = IK_5_link(x, y)
thetaR = res[0].x[0]
thetaL = res[0].x[1]

J_current = J(thetaR,thetaL)

u, s, v = np.linalg.svd(J_current)

ax.cla()
draw_robot(thetaR, thetaL, ax = ax)

ang = 180*np.arctan2(u[1,0], u[0,0])/np.pi
ell = mpl.patches.Ellipse(xy=(x,y), width=s[0], height=s[1], angle = ang, alpha = 0.5)
ax.add_patch(ell)

plt.quiver(x, y, J_current[0,0], J_current[1,0], scale = 1, color = 'r')

plt.quiver(x, y, J_current[0,1], J_current[1,1], scale = 1, color = 'b')


plt.show()
plt.draw()

plt.title("theta_R = {:.4f}, theta_L = {:.4f}, err = {:.4f}".format(thetaR, thetaL, res[1]))


## Compute quantities

In [28]:
xx = np.linspace(-.3, .3, 100)
yy = np.linspace(.0, .35, 50)

force_x = np.zeros((yy.shape[0], xx.shape[0]))
force_y = np.zeros((yy.shape[0], xx.shape[0]))

proprioception = np.zeros((yy.shape[0], xx.shape[0]))
force_production = np.zeros((yy.shape[0], xx.shape[0]))
workspace = np.zeros((yy.shape[0], xx.shape[0]))

for k, x in enumerate(xx):
    for j, y in enumerate(yy):

        res = IK_5_link(x, y)
        thetaR = res[0].x[0]
        thetaL = res[0].x[1]
        
        if res[0].success == False:
            continue
        
        workspace[j,k] = 1
        
        J_current = J(thetaR,thetaL)

        u, s, v = np.linalg.svd(J_current)
        
        proprioception[j, k] = s[1]
        force_production[j, k] = s[0]
        
        force = K * (np.array([[1, -1]]) @ np.linalg.inv(J_current)).T
        
        force_y[j, k] = force[1]
        force_x[j, k] = force[0]
        
#     print(k)

In [ ]:
xx = np.linspace(-.135, .135, 100)
yy = np.linspace(.0, .25, 50)

force_x = np.zeros((yy.shape[0], xx.shape[0]))
force_y = np.zeros((yy.shape[0], xx.shape[0]))

workspace = np.zeros((yy.shape[0], xx.shape[0]))

for k, x in enumerate(xx):
    for j, y in enumerate(yy):

        res = IK_5_link(x, y)
        thetaR = res[0].x[0]
        thetaL = res[0].x[1]
        
        if res[0].success == False:
            continue
        
        workspace[j,k] = 1
        
    print(k)

In [29]:

plt.clf()
plt.imshow(workspace)

## Compute peak vertical force / mass


In [ ]:
yy = np.linspace(.06, .25, 50)
force_y = np.zeros((yy.shape[0],1))

for j, y in enumerate(yy):
    res = IK_5_link(x, y)
    thetaR = res[0].x[0]
    thetaL = res[0].x[1]

    if res[1] > 0.0001: 
        continue

    J_current = J(thetaR,thetaL)

    force = K * (np.array([[1, -1]]) @ np.linalg.inv(J_current)).T
    
    force_y[j] = force[1]
    
    print(j)
    


In [ ]:
plt